In [2]:
from path_to_folders import images_path, labels_path, masks_path
import tensorflow as tf


### Define variables

In [ ]:
CLASSES = 2
COLOR_MAP = {
    0: 'magenta',
    1: 'yellow'
}

SAMPLE_SIZE = (384, 384)
OUTPUT_SIZE = (768, 768)